# Recommendation System using MiniBatchKMeans


In [1]:
#Import necessary libraries
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import time
import math

In [2]:
# Analysing the input dataframe
df = pd.read_csv("../netflix_titles.csv")


In [3]:
# mapping the multiple value cells into list type
df["directors"] = df["director"].apply(
    lambda l: [] if pd.isna(l) else [i.strip().replace(" ", "_") for i in l.split(",")]
)
df["genres"] = df["listed_in"].apply(
    lambda l: [] if pd.isna(l) else [i.strip().replace(" ", "_") for i in l.split(",")]
)
df["actors"] = df["cast"].apply(
    lambda l: [] if pd.isna(l) else [i.strip().replace(" ", "_") for i in l.split(",")]
)
df["countries"] = df["country"].apply(
    lambda l: [] if pd.isna(l) else [i.strip().replace(" ", "_") for i in l.split(",")]
)
df["all_features"] = df["directors"] + df["genres"] + df["actors"] + df["countries"]
df["all_features"] = df["all_features"].apply(lambda x: " ".join(x))


In [4]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [5]:
def text_cleanser(text):
    stemmer = WordNetLemmatizer()
    text = "".join(
        [char for char in text if (char.isalpha() or char.isspace()) and char != "'"]
    )
    text = [
        word.lower()
        for word in text.split()
        if word.lower() not in stopwords.words("english")
    ]
    return [stemmer.lemmatize(word) for word in text]


In [6]:
df_desc = df[["show_id", "description"]].set_index("show_id")
df_desc["trunc_desc"] = df_desc["description"].apply(lambda x: text_cleanser(x))


In [7]:
vocabulary = {}
for words in df_desc.trunc_desc:
    for word in words:
        if word in vocabulary:
            vocabulary[word] = vocabulary[word] + 1
        else:
            vocabulary[word] = 1
df_voc = pd.DataFrame(list(vocabulary.items()), columns=["text", "count"])


#### Clustering :-


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

bow_transformer = CountVectorizer(analyzer=text_cleanser).fit(df_desc["description"])
desc_bow = bow_transformer.transform(df_desc["description"])
print("Shape of the generated matrix : ", desc_bow.shape)
sparsity = desc_bow.nnz / (desc_bow.shape[0] * desc_bow.shape[1]) * 100.0
print("Sparsity of the generated matrix", round(sparsity, 4))


Shape of the generated matrix :  (8807, 18085)
Sparsity of the generated matrix 0.079


In [9]:
# Finding TF and IDF metrics
vector = TfidfVectorizer(
    max_df=1,
    min_df=1,
    strip_accents="ascii",
    stop_words="english",
    lowercase=True,
    use_idf=True,
    norm="l2",
    smooth_idf=True,
)
tfidf = vector.fit_transform(df.all_features)


In [10]:
# Cluster the description data using MiniBatchKMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import linear_kernel

# setting the no. of resulting clusters for kmeans
k = 400
kmeans = MiniBatchKMeans(n_clusters=k, init="k-means++")
kmeans.fit(tfidf)
centers = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vector.get_feature_names_out()
request_transform = vector.transform(df["all_features"])
# new column cluster based on the description
df["cluster"] = kmeans.predict(request_transform)


In [11]:
def find_similar_movies(tfidf_matrix, index, top_n=5):
    cosine_similarities = linear_kernel(
        tfidf_matrix[index : index + 1], tfidf_matrix
    ).flatten()
    related_docs_indices = [
        i for i in cosine_similarities.argsort()[::-1] if i != index
    ]
    return [index for index in related_docs_indices][0:top_n]


## Graph preparation

- Insert all the notable parameters like castings,genre and directors as nodes in the graph.
- use find_similar_movies() method to fetch the n related movies for the given node
- Connect the all the related movie nodes with the selected movie with an edge names 'SIMILAR'


In [12]:
G = nx.Graph(label="MOVIE")
start_time = time.time()
for i, rowi in df.iterrows():
    G.add_node(
        rowi["title"],
        key=rowi["show_id"],
        label="MOVIE",
        mtype=rowi["type"],
        rating=rowi["rating"],
    )
    for element in rowi["actors"]:
        G.add_node(element, label="PERSON")
        G.add_edge(rowi["title"], element, label="ACTED_IN")
    for element in rowi["genres"]:
        G.add_node(element, label="GENRE")
        G.add_edge(rowi["title"], element, label="GENRE_IN")
    for element in rowi["directors"]:
        G.add_node(element, label="PERSON")
        G.add_edge(rowi["title"], element, label="DIRECTED")
    for element in rowi["countries"]:
        G.add_node(element, label="COU")
        G.add_edge(rowi["title"], element, label="COU_IN")

    indices = find_similar_movies(tfidf, i, top_n=3)
    snode = "Sim(" + rowi["title"][:15].strip() + ")"
    G.add_node(snode, label="SIMILAR")
    G.add_edge(rowi["title"], snode, label="SIMILARITY")
    for element in indices:
        G.add_edge(snode, df["title"].loc[element], label="SIMILARITY")
print(" finish -- {} seconds --".format(time.time() - start_time))


 finish -- 51.47497630119324 seconds --


In [13]:
def get_all_adj_nodes(list_in):
    sub_graph = set()
    for m in list_in:
        sub_graph.add(m)
        for e in G.neighbors(m):
            sub_graph.add(e)
    return list(sub_graph)


def draw_sub_graph(sub_graph, title=""):
    subgraph = G.subgraph(sub_graph)
    colors = []
    sizes = []
    std_size = 500
    for e in subgraph.nodes():
        if G.nodes[e]["label"] == "MOVIE":
            colors.append("#b3042c")
            sizes.append(std_size * 5)
        elif G.nodes[e]["label"] == "PERSON":
            colors.append("#047d59")
            sizes.append(std_size * 2.5)
        elif G.nodes[e]["label"] == "GENRE":
            colors.append("#3a018a")
            sizes.append(std_size)
        elif G.nodes[e]["label"] == "COU":
            colors.append("#bd3102")
            sizes.append(std_size * 0.9)
        elif G.nodes[e]["label"] == "SIMILAR":
            colors.append("#b87906")
            sizes.append(std_size)
        elif G.nodes[e]["label"] == "CLUSTER":
            colors.append("#cdf7e9")
            sizes.append(std_size * 0.9)
    fig, ax = plt.subplots(figsize=(18, 18))
    nx.draw(
        subgraph,
        with_labels=True,
        alpha=0.8,
        node_shape="o",
        node_size=sizes,
        cmap="Accent",
        edge_color="white",
        font_color="white",
        font_weight="bold",
        node_color=colors,
    )
    # plt.title('Recommendations after watching The video '+title+' :-')
    ax.set_title("Recommendations for Movie " + title + " :-", color="white", size=30)
    ax.axis("off")
    fig.set_facecolor("#44495c")
    plt.tight_layout()
    plt.gcf().set_dpi(400)
    plt.show()


In [14]:
def get_recommendation(root):
    commons_dict = {}
    for e in G.neighbors(root):
        for e2 in G.neighbors(e):
            if e2 == root:
                continue
            if G.nodes[e2]["label"] == "MOVIE":
                commons = commons_dict.get(e2)
                if commons == None:
                    commons_dict.update({e2: [e]})
                else:
                    commons.append(e)
                    commons_dict.update({e2: commons})
    movies = []
    weight = []
    for key, values in commons_dict.items():
        w = 0.0
        for e in values:
            w = w + 1 / math.log(G.degree(e))
        movies.append(key)
        weight.append(w)

    result = pd.Series(data=np.array(weight), index=movies)
    result.sort_values(inplace=True, ascending=False)
    return result


In [15]:
Movie = "Dark"
recommendation = get_recommendation(Movie)
rel_graph = get_all_adj_nodes([Movie] + list(recommendation.index[:2]))
draw_sub_graph(rel_graph, Movie)
